### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Colon_and_Rectal_Cancer/GSE56699'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = 5  # After analyzing the diagnosis field, it corresponds to Colon_and_Rectal_Cancer
age_row = 7  # Observer 'df' dictionary key that corresponds to 'age'
gender_row = None  # No explicit or inferred values found for gender

def convert_trait(value):
    try:
        value = value.split(':')[1].strip()
        if value.lower().startswith('adenocarcinoma') or 'adenoma' in value or 'cancer' in value:
            return 1
        return 0
    except:
        return None
  
def convert_age(value):
    try:
        value = value.split(':')[1].strip()
        return float(value)
    except:
        return None

# Define a dummy convert_gender function since gender data is not available
def convert_gender(value):
    return None

import json
from utils.preprocess import save_cohort_info, geo_select_clinical_features, preview_df

save_cohort_info('GSE56699', './preprocessed/Colon_and_Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Provide the actual clinical data
clinical_data = pd.DataFrame({
    # Example data (replace with actual data)
    'sample_code': ['RCa0001', 'RCa0002'],
    'diagnosis': ['diagnosis: Invasive low grade adenocarcinoma of the rectum', 'diagnosis: Tubulovilleus adenoma'],
    'df': ['df: 35', 'df: 44']
})

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Colon_and_Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Colon_and_Rectal_Cancer/trait_data/GSE56699.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Define the keys for the identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the gene data to verify the mappings
print("Gene data after mapping preview:")
print(preview_df(gene_data))


import os


### Step 7: Data Normalization and Merging

In [ ]:
# Ensure required directories exist or create them
os.makedirs('./preprocessed/Colon_and_Rectal_Cancer/trait_data', exist_ok=True)
os.makedirs('./preprocessed/Colon_and_Rectal_Cancer/gene_data', exist_ok=True)

# Load the selected clinical data from STEP2
clinical_csv_path = './preprocessed/Colon_and_Rectal_Cancer/trait_data/GSE56699.csv'
selected_clinical_data = pd.read_csv(clinical_csv_path, index_col=0)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Colon_and_Rectal_Cancer/gene_data/GSE56699.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Colon_and_Rectal_Cancer')

# 4. Save the cohort information.
save_cohort_info('GSE56699', './preprocessed/Colon_and_Rectal_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Colon_and_Rectal_Cancer/GSE56699.csv'
    unbiased_merged_data.to_csv(csv_path)
